In [ ]:
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "0"

# Carregando Pacotes

In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from matplotlib import pyplot as plt
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorboard.plugins.hparams import api as hp
from pathlib import Path
import sklearn.metrics
import sklearn
import itertools
import io
import tensorflow_addons as tfa

2022-03-31 08:22:44.673035: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
print(tf.__version__)

2.4.1


# Paramêtros

In [ ]:
# Classes
num_classes = 12

# Epochs
n_epochs = 200
n_epochs_ini = 100

# Tamanho imagem
img_rows, img_cols = 224, 224

input_shape = (img_rows, img_cols, 3)

#batch_size = 64

strategy = tf.distribute.MirroredStrategy()

BATCH_SIZE_PER_REPLICA = 16
batch_size = BATCH_SIZE_PER_REPLICA * strategy.num_replicas_in_sync

options = tf.data.Options()
options.experimental_distribute.auto_shard_policy = tf.data.experimental.AutoShardPolicy.DATA


# seed
seed = 42

path_dataset = Path('/scratch/dpfurtado/pilheira_new_home/transfer learning/2_train_train_dev_test/224_224/')
path_train = path_dataset/'train'
path_train_dev = path_dataset/'train-dev'
path_test = path_dataset/'test_archived'

nome_modelo = 'resnet101_plc_full'


logs_dir = os.path.join('./logs/', nome_modelo)

!rm -r {logs_dir}

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')


2022-03-31 08:22:46.606431: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2022-03-31 08:22:46.608751: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-03-31 08:22:47.654399: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:3b:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-03-31 08:22:47.655311: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 1 with properties: 
pciBusID: 0000:af:00.0 name: NVIDIA GeForce GTX TITAN Black computeCapability: 3.5
coreClock: 0.98GHz coreCount: 15 deviceMemorySize: 5.94GiB deviceMemoryBandwidth: 312.92GiB/s
2022-03-31 08:22:47.655426: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcu

# Carregando Base de Dados

In [ ]:
train_ds = tf.data.Dataset.list_files(str(path_train/'*/*'), shuffle=False)
train_dev_ds_concat = tf.data.Dataset.list_files(str(path_train_dev/'*/*'), shuffle=False)
test_ds = tf.data.Dataset.list_files(str(path_test/'*/*'), shuffle=False)

In [ ]:
train_ds = train_ds.concatenate(train_dev_ds_concat)

In [ ]:
len_train = tf.data.experimental.cardinality(train_ds).numpy()
print(len_train)

95993


In [ ]:
class_names = np.array(sorted([item.name for item in path_train.glob('*')]))
print(class_names)

['Acrop' 'Fav' 'Favia' 'HC_other' 'Macro' 'Mille' 'Monti' 'Pavon' 'Plat'
 'Pocill' 'Porit' 'Soft']


In [ ]:
len_train = tf.data.experimental.cardinality(train_ds).numpy()
len_test = tf.data.experimental.cardinality(test_ds).numpy()
print(len_train)
print(len_test)

95993
3130


In [ ]:
def get_label(file_path):
  # Convert the path to a list of path components
  parts = tf.strings.split(file_path, os.path.sep)
  # The second to last is the class-directory
  one_hot = parts[-2] == class_names
  # Integer encode the label
  return tf.argmax(one_hot)

In [ ]:
def decode_img(img):
    # Convert the compressed string to a 3D uint8 tensor
    img = tf.io.decode_jpeg(img, channels=3)
    # Resize the image to the desired size
    #return tf.image.resize(img, [img_height, img_width])
    return img

In [ ]:
def process_path(file_path):
    label = get_label(file_path)
    # Load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE 
train_ds = train_ds.map(tf.autograph.experimental.do_not_convert(process_path), num_parallel_calls=AUTOTUNE)
test_ds = test_ds.map(tf.autograph.experimental.do_not_convert(process_path), num_parallel_calls=AUTOTUNE)

In [ ]:
def configure_for_performance_train(ds):
    ds = ds.cache()
    ds = ds.shuffle(buffer_size=len_train)
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds
def configure_for_performance_test(ds):
    ds = ds.cache()
    ds = ds.batch(batch_size)
    ds = ds.prefetch(buffer_size=AUTOTUNE)
    return ds

train_dataset = configure_for_performance_train(train_ds)
test_dataset = configure_for_performance_test(test_ds)

In [ ]:
train_dataset = train_dataset.with_options(options)
test_dataset = test_dataset.with_options(options)

In [ ]:
y_test = np.concatenate([y for x, y in test_dataset], axis=0)

2022-03-31 08:22:50.909028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2022-03-31 08:22:50.931579: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2200000000 Hz


In [ ]:
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.

    Args:
      cm (array, shape = [n, n]): a confusion matrix of integer classes
      class_names (array, shape = [n]): String names of the integer classes
    """
    figure = plt.figure(figsize=(8, 8))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title("Confusion matrix")
    plt.colorbar()
    tick_marks = np.arange(len(class_names))
    plt.xticks(tick_marks, class_names, rotation=90)
    plt.yticks(tick_marks, class_names)

    # Compute the labels from the normalized confusion matrix.
    labels = np.around(cm, decimals=2)

    # Use white text if squares are dark; otherwise black.
    threshold = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        color = "white" if cm[i, j] > threshold else "black"
        plt.text(j, i, labels[i, j], horizontalalignment="center", color=color)

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    return figure


def plot_to_image(figure):
    """Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call."""
    # Save the plot to a PNG in memory.
    buf = io.BytesIO()
    plt.savefig(buf, format='png')
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    # Convert PNG buffer to TF image
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    # Add the batch dimension
    image = tf.expand_dims(image, 0)
    return image


In [ ]:
run_dir = logs_dir

file_writer_cm = tf.summary.create_file_writer(run_dir + '/cm')


def log_confusion_matrix(epoch, logs):
    # Use the model to predict the values from the validation dataset.
    test_pred_raw = model.predict(test_dataset)
    test_pred = np.argmax(test_pred_raw, axis=1)

    # Calculate the confusion matrix.
    cm = sklearn.metrics.confusion_matrix(
        y_test, test_pred, normalize='true')
    # Log the confusion matrix as an image summary.
    figure = plot_confusion_matrix(cm, class_names=class_names)
    cm_image = plot_to_image(figure)

    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)


# Define the per-epoch callback.
cm_callback = keras.callbacks.LambdaCallback(
    on_epoch_end=log_confusion_matrix)
with strategy.scope():
    # loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # métricas
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
    data_augmentation = tf.keras.Sequential(
        [tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal_and_vertical')])
    # Keras sequential model with Hyperparameters passed from the argument
    cnn = ResNet101(weights='imagenet', include_top=False,
                    input_shape=input_shape)
    # Para o primeiro treinamento as camadas convolucionais não serão treinadas
    cnn.trainable = False
    inputs = keras.Input(shape=input_shape)
    x = data_augmentation(inputs)
    x = preprocess_input(x)
    # Camadas de Batch Normalization em inference mode
    x = cnn(x, training=False)
    x = GlobalAveragePooling2D()(x)
    x = keras.layers.Dropout(0.2)(x)
    predictions = Dense(num_classes)(x)
    model = Model(inputs, predictions)
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=1e-4, momentum=0.9), loss=loss, metrics=metrics)


def scheduler_denso(epoch, lr):
    lr_0 = 1e-4
    epoch_ini_lr = 20
    if epoch < epoch_ini_lr:
        learning_rate = lr
        with tf.summary.create_file_writer(run_dir + '/metric').as_default():
            tf.summary.scalar(
                'learning rate', data=learning_rate, step=epoch)
        return learning_rate
    else:
        learning_rate = lr_0 * \
            tf.math.exp(-0.005*(epoch-epoch_ini_lr)).numpy()
        with tf.summary.create_file_writer(run_dir + '/metric').as_default():
            tf.summary.scalar(
                'learning rate', data=learning_rate, step=epoch)
        return learning_rate


callbacks_dense = [tf.keras.callbacks.TensorBoard(run_dir, profile_batch=0, write_graph=False),
                   tf.keras.callbacks.LearningRateScheduler(
    scheduler_denso, verbose=0),
    cm_callback,
    EarlyStopping(patience=10)]
# Training the network
history_dense = model.fit(train_dataset, epochs=n_epochs_ini, verbose=0,
                          validation_data=test_dataset, callbacks=callbacks_dense)

treinamento_denso = len(history_dense.history['loss'])

In [ ]:
with strategy.scope():
    cnn.trainable = True
    # loss
    loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    # métricas
    metrics = [tf.keras.metrics.SparseCategoricalAccuracy(name='accuracy')]
    model.compile(
        optimizer=keras.optimizers.SGD(
            learning_rate=1e-5, momentum=0.9),
        loss=loss,
        metrics=metrics
    )


def scheduler_cnn(epoch, lr):
    lr_0 = 1e-5
    learning_rate = lr_0 * \
        tf.math.exp(-0.02*(epoch-treinamento_denso)).numpy()
    with tf.summary.create_file_writer(run_dir + '/metric').as_default():
        tf.summary.scalar(
            'learning rate', data=learning_rate, step=epoch)
    return learning_rate


nome_save_model = nome_modelo

model_dir = os.path.join('./model/', nome_save_model)

callbacks_cnn = [tf.keras.callbacks.TensorBoard(run_dir, profile_batch=0, write_graph=False),
                 tf.keras.callbacks.LearningRateScheduler(
                 scheduler_cnn, verbose=0),
                 cm_callback,
                 EarlyStopping(patience=20),
                 ModelCheckpoint(filepath=model_dir, monitor="val_loss", save_best_only=True)]

history_cnn = model.fit(train_dataset, epochs=n_epochs, initial_epoch=treinamento_denso, verbose=0,
                        validation_data=test_dataset, callbacks=callbacks_cnn)

treinamento_cnn = len(history_cnn.history['loss'])

In [ ]:
print(treinamento_denso)
print(treinamento_cnn)

43
59
